# Performance Variability Boxplots

Performance variability boxplots provide an insight into the runtime distribution and its varibility across callsites. Boxplots are calculated to represent the range of the distribution and outliers (dots) correspond which are beyond the 1.5*IQR. Additionally, several statistical measures like mean, variance, kurtosis, skewness are also provided.

In [1]:
import os, sys
from IPython.display import HTML, display

# Hatchet imports
import hatchet as ht
from hatchet.util.unify_ensemble import unify_ensemble
from hatchet.util.boxplot import BoxPlot

First, we will construct a **hatchet.GraphFrame** using a sample dataset in our repository, **caliper-lulesh-json**. 

In [2]:
data_dir = os.path.realpath("../../../hatchet/tests/data")
data_path = os.path.join(data_dir, "caliper-lulesh-json/lulesh-annotation-profile.json")

gf_list = []
for i in range(10):
    gf = ht.GraphFrame.from_caliper(data_path)
    gf.dataset = "dset{}".format(i)
    gf_list.append(gf)

gf_ensemble = unify_ensemble(gf_list)

In [3]:
gf_ensemble.dataframe

name  nid  \
node                                        rank dataset                       
{'name': 'main', 'type': 'region'}          0    dset9             main    0   
                                            4    dset7             main    0   
                                            5    dset7             main    0   
                                            6    dset7             main    0   
                                            7    dset7             main    0   
...                                                                 ...  ...   
{'name': 'TimeIncrement', 'type': 'region'} 1    dset3    TimeIncrement   12   
                                            2    dset3    TimeIncrement   12   
                                            3    dset3    TimeIncrement   12   
                                                 dset5    TimeIncrement   12   
                                            7    dset0    TimeIncrement   12   

                                                              time  \
node                                        rank dataset             
{'name': 'main', 'type': 'region'}          0    dset9    121489.0   
                                            4    dset7    118953.0   
                                            5    dset7    133256.0   
                                            6    dset7    114035.0   
                                            7    dset7    137098.0   
...                                                            ...   
{'name': 'TimeIncrement', 'type': 'region'} 1    dset3    212402.0   
                                            2    dset3    171635.0   
                                            3    dset3    323519.0   
                                                 dset5    323519.0   
                                            7    dset0       540.0   

                                                          time (inc)  \
node                                        rank dataset               
{'name': 'main', 'type': 'region'}          0    dset9     5882425.0   
                                            4    dset7     5905595.0   
                                            5    dset7     5877613.0   
                                            6    dset7     5870933.0   
                                            7    dset7     5898724.0   
...                                                              ...   
{'name': 'TimeIncrement', 'type': 'region'} 1    dset3      212402.0   
                                            2    dset3      171635.0   
                                            3    dset3      323519.0   
                                                 dset5      323519.0   
                                            7    dset0         540.0   

                                                          hatchet_nid  
node                                        rank dataset               
{'name': 'main', 'type': 'region'}          0    dset9              0  
                                            4    dset7              0  
                                            5    dset7              0  
                                            6    dset7              0  
                                            7    dset7              0  
...                                                               ...  
{'name': 'TimeIncrement', 'type': 'region'} 1    dset3             23  
                                            2    dset3             23  
                                            3    dset3             23  
                                                 dset5             23  
                                            7    dset0             23  

[1920 rows x 5 columns]

Next, using the **hatchet.GraphFrame**, we can calculate the data required for performance variability boxplot using an exposed hatchet API, **Boxplot**.

The interface excepts the following attributes:
1. `tgt_gf` - Target hatchet.GraphFrame 
2. `bkg_gf` - Background hatchet.GraphFrame (optional)
3. `callsites` - List of callsite names for which we want to compute/visualize the boxplots.
4. `metrics` - Runtime metrics for which we need to calculate the boxplots.
5. `iqr_scale` - Interquartile range scale (by default = 1.5)

In [13]:
bp = BoxPlot(multi_index_gf=gf_ensemble, metrics=["time"], drop_index="rank")

/Users/jarus/Work/llnl/hatchet/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (await self.run_code(code, result,  async_=asy)):


**Boxplot** API calculates the results and stores as a GraphFrames in a dictionary (i.e., `tgt` and `bkg`). 

In [14]:
bp.gf['time'].dataframe

hatchet_nid  \
node                                        dataset                
{'name': 'main', 'type': 'region'}          dset9              0   
                                            dset7              0   
                                            dset1              0   
                                            dset3              0   
                                            dset6              0   
...                                                          ...   
{'name': 'TimeIncrement', 'type': 'region'} dset8             23   
                                            dset9             23   
                                            dset2             23   
                                            dset3             23   
                                            dset4             23   

                                                                                                     q  \
node                                        dataset                                                      
{'name': 'main', 'type': 'region'}          dset9    [105528.0, 113072.25, 116494.0, 124430.75, 137...   
                                            dset7    [105528.0, 113072.25, 116494.0, 124430.75, 137...   
                                            dset1    [105528.0, 113072.25, 116494.0, 124430.75, 137...   
                                            dset3    [105528.0, 113072.25, 116494.0, 124430.75, 137...   
                                            dset6    [105528.0, 113072.25, 116494.0, 124430.75, 137...   
...                                                                                                ...   
{'name': 'TimeIncrement', 'type': 'region'} dset8     [540.0, 202210.25, 269561.0, 361367.0, 423809.0]   
                                            dset9     [540.0, 202210.25, 269561.0, 361367.0, 423809.0]   
                                            dset2     [540.0, 202210.25, 269561.0, 361367.0, 423809.0]   
                                            dset3     [540.0, 202210.25, 269561.0, 361367.0, 423809.0]   
                                            dset4     [540.0, 202210.25, 269561.0, 361367.0, 423809.0]   

                                                          min       max  \
node                                        dataset                       
{'name': 'main', 'type': 'region'}          dset9    105528.0  137098.0   
                                            dset7    105528.0  137098.0   
                                            dset1    105528.0  137098.0   
                                            dset3    105528.0  137098.0   
                                            dset6    105528.0  137098.0   
...                                                       ...       ...   
{'name': 'TimeIncrement', 'type': 'region'} dset8       540.0  423809.0   
                                            dset9       540.0  423809.0   
                                            dset2       540.0  423809.0   
                                            dset3       540.0  423809.0   
                                            dset4       540.0  423809.0   

                                                          mean           var  \
node                                        dataset                            
{'name': 'main', 'type': 'region'}          dset9    119373.50  1.044980e+08   
                                            dset7    119373.50  1.044980e+08   
                                            dset1    119373.50  1.044980e+08   
                                            dset3    119373.50  1.044980e+08   
                                            dset6    119373.50  1.044980e+08   
...                                                        ...           ...   
{'name': 'TimeIncrement', 'type': 'region'} dset8    263538.75  1.775294e+10   
                                            dset9    263538.75  1.775294e+10   
             

In [6]:
bp.gf['time'].dataframe.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 240 entries, (Node({'name': 'main', 'type': 'region'}), 'dset9') to (Node({'name': 'TimeIncrement', 'type': 'region'}), 'dset4')
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hatchet_nid  240 non-null    int64  
 1   q            240 non-null    object 
 2   min          240 non-null    float64
 3   max          240 non-null    float64
 4   mean         240 non-null    float64
 5   var          240 non-null    float64
 6   imb          240 non-null    float64
 7   kurt         240 non-null    float64
 8   skew         240 non-null    float64
 9   name         240 non-null    object 
dtypes: float64(7), int64(1), object(2)
memory usage: 20.4+ KB


In [15]:
bp = BoxPlot(multi_index_gf=gf_ensemble, metrics=["time"], drop_index="dataset")

/Users/jarus/Work/llnl/hatchet/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: PerformanceWarning: indexing past lexsort depth may impact performance.
  if (await self.run_code(code, result,  async_=asy)):


In [16]:
bp.gf['time'].dataframe

hatchet_nid  \
node                                        rank                
{'name': 'main', 'type': 'region'}          0               0   
                                            4               0   
                                            5               0   
                                            6               0   
                                            7               0   
...                                                       ...   
{'name': 'TimeIncrement', 'type': 'region'} 2              23   
                                            1              23   
                                            0              23   
                                            7              23   
                                            6              23   

                                                                                                  q  \
node                                        rank                                                      
{'name': 'main', 'type': 'region'}          0     [121489.0, 121489.0, 121489.0, 121489.0, 12148...   
                                            4     [118953.0, 118953.0, 118953.0, 118953.0, 11895...   
                                            5     [133256.0, 133256.0, 133256.0, 133256.0, 13325...   
                                            6     [114035.0, 114035.0, 114035.0, 114035.0, 11403...   
                                            7     [137098.0, 137098.0, 137098.0, 137098.0, 13709...   
...                                                                                             ...   
{'name': 'TimeIncrement', 'type': 'region'} 2     [171635.0, 171635.0, 171635.0, 171635.0, 17163...   
                                            1     [212402.0, 212402.0, 212402.0, 212402.0, 21240...   
                                            0     [418469.0, 418469.0, 418469.0, 418469.0, 41846...   
                                            7                   [540.0, 540.0, 540.0, 540.0, 540.0]   
                                            6     [423809.0, 423809.0, 423809.0, 423809.0, 42380...   

                                                       min       max  \
node                                        rank                       
{'name': 'main', 'type': 'region'}          0     121489.0  121489.0   
                                            4     118953.0  118953.0   
                                            5     133256.0  133256.0   
                                            6     114035.0  114035.0   
                                            7     137098.0  137098.0   
...                                                    ...       ...   
{'name': 'TimeIncrement', 'type': 'region'} 2     171635.0  171635.0   
                                            1     212402.0  212402.0   
                                            0     418469.0  418469.0   
                                            7        540.0     540.0   
                                            6     423809.0  423809.0   

                                                      mean  var  imb  kurt  \
node                                        rank                             
{'name': 'main', 'type': 'region'}          0     121489.0  0.0  0.0  -3.0   
                                            4     118953.0  0.0  0.0  -3.0   
                                            5     133256.0  0.0  0.0  -3.0   
                                            6     114035.0  0.0  0.0  -3.0   
                                            7     137098.0  0.0  0.0  -3.0   
...                                                    ...  ...  ...   ...   
{'name': 'TimeIncrement', 'type': 'region'} 2     171635.0  0.0  0.0  -3.0   
                                            1     212402.0  0.0  0.0  -3.0   
                                            0     418469.0  0.0  0.0  -3.0   
                                            7        540.0  0.0  0.0 

Using the **roundtrip** interface, we can then visualize the compute boxplot information. Below, we load the roundtrip interface that allows users to visualize plots on jupyter notebook cells directly. 

In [8]:
bp_json = bp.to_json()

In [9]:
bp_json

{(0,
  'dset9'): {'time': {'q': [105528.0,
    113072.25,
    116494.0,
    124430.75,
    137098.0],
   'min': 105528.0,
   'max': 137098.0,
   'mean': 119373.5,
   'var': 104497970.25,
   'imb': 0.14847935262013764,
   'kurt': -0.9421848873183336,
   'skew': 0.5436725364039101,
   'name': 'main',
   'node': Node({'name': 'main', 'type': 'region'}),
   'rank': 0}},
 (0,
  'dset7'): {'time': {'q': [105528.0,
    113072.25,
    116494.0,
    124430.75,
    137098.0],
   'min': 105528.0,
   'max': 137098.0,
   'mean': 119373.5,
   'var': 104497970.25,
   'imb': 0.14847935262013764,
   'kurt': -0.9421848873183336,
   'skew': 0.5436725364039101,
   'name': 'main',
   'node': Node({'name': 'main', 'type': 'region'}),
   'rank': 4}},
 (0,
  'dset1'): {'time': {'q': [105528.0,
    113072.25,
    116494.0,
    124430.75,
    137098.0],
   'min': 105528.0,
   'max': 137098.0,
   'mean': 119373.5,
   'var': 104497970.25,
   'imb': 0.14847935262013764,
   'kurt': -0.9421848873183336,
   'skew': 0